In [ ]:
workdir_ = '/glade/work/juliob/'
import sys
sys.path.append(workdir_ + 'myPythonTools/GenlTools/')
sys.path.append(workdir_ + 'myPythonTools/Utils/')
sys.path.append(workdir_ + 'PyRegridding/Regridder/')
sys.path.append(workdir_ + 'PyRegridding/Utils/')

# Own local packages
import AveragingUtils as Av
import VertRegridFlexLL as Vrg
import PlotUtil as Pu
import validation_data as Val
import DatasetMngmt as DsMn

# The usual
from datetime import date
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib.tri as tri
from matplotlib.colors import ListedColormap, BoundaryNorm

# Cartopy for pretty maps
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# Some other useful packages 
import importlib
import copy
import time
import cftime
import glob


importlib.reload( Pu )
importlib.reload(Av)
importlib.reload(Val)
importlib.reload(DsMn)


In [ ]:
%%time

exp_C = 'fmtndg2000_MM_control'  # 'fmtndg_S2_x04' #_rd1p0_cd0p1'
#exp_X = 'fmtndg2000_MM_NOblj'  # 'fmtndg_S2_x04' #_rd1p0_cd0p1'
exp_X = 'fmtndg2000_MM_x21_2'  # 'fmtndg_S2_x04' #_rd1p0_cd0p1'
subd='regridded'
read_npsave=True

if (read_npsave==False):
    # Example with glob - this assumes files are in the current directory
    file_paths_C3 = []
    file_paths_C2 = []
    file_paths_C1 = []
    file_paths_X3 = []
    file_paths_X2 = []
    file_paths_X1 = []
    months = ['07', '08' , '09' ]
    for month in months:
        htap='cam.h3'
        path = f"/glade/derecho/scratch/juliob/archive/{exp_C}/atm/{subd}/{exp_C}.{htap}.2001-{month}-*.nc" 
        file_paths_C3.extend(glob.glob(path))
        path = f"/glade/derecho/scratch/juliob/archive/{exp_X}/atm/{subd}/{exp_X}.{htap}.2001-{month}-*.nc" 
        file_paths_X3.extend(glob.glob(path))
    
        htap='cam.h2'
        path = f"/glade/derecho/scratch/juliob/archive/{exp_C}/atm/{subd}/{exp_C}.{htap}.2001-{month}-*.nc" 
        file_paths_C2.extend(glob.glob(path))
        path = f"/glade/derecho/scratch/juliob/archive/{exp_X}/atm/{subd}/{exp_X}.{htap}.2001-{month}-*.nc" 
        file_paths_X2.extend(glob.glob(path))
    
        htap='cam.h1'
        path = f"/glade/derecho/scratch/juliob/archive/{exp_C}/atm/{subd}/{exp_C}.{htap}.2001-{month}-*.nc" 
        file_paths_C1.extend(glob.glob(path))
        path = f"/glade/derecho/scratch/juliob/archive/{exp_X}/atm/{subd}/{exp_X}.{htap}.2001-{month}-*.nc" 
        file_paths_X1.extend(glob.glob(path))
    
    # Sort the file paths to ensure they are in order
    file_paths_C3 = sorted(file_paths_C3)
    file_paths_C2 = sorted(file_paths_C2)
    file_paths_C1 = sorted(file_paths_C1)
    file_paths_X3 = sorted(file_paths_X3)
    file_paths_X2 = sorted(file_paths_X2)
    file_paths_X1 = sorted(file_paths_X1)
    #And get the data
    D_C1 = xr.open_mfdataset( file_paths_C1 ,data_vars='different', coords='different' )
    D_X1 = xr.open_mfdataset( file_paths_X1 ,data_vars='different', coords='different' )
    
    D_C2 = xr.open_mfdataset( file_paths_C2 ,data_vars='different', coords='different' )
    D_X2 = xr.open_mfdataset( file_paths_X2 ,data_vars='different', coords='different' )
    
    D_C3 = xr.open_mfdataset( file_paths_C3 ,data_vars='different', coords='different' )
    D_X3 = xr.open_mfdataset( file_paths_X3 ,data_vars='different', coords='different' )

    #Extract and save
    Dic_C = DsMn.extract_save( X1=D_C1 , X2=D_C2 ,X3=D_C3,exp_X=exp_C , read=read_npsave )
    Dic_X = DsMn.extract_save( X1=D_X1 , X2=D_X2 ,X3=D_X3,exp_X=exp_X , read=read_npsave )
else:
    Dic_C = DsMn.extract_save( exp_X=exp_C , read=read_npsave )
    Dic_X = DsMn.extract_save( exp_X=exp_X , read=read_npsave )



In [ ]:
print(Dic_X.files)

In [ ]:
%%time
#######################################


uu_C = Dic_C['U']
vv_C = Dic_C['V']
utn_C = Dic_C['Nudge_U']
prl_C =  Dic_C['PRECL']
stend_C = Dic_C['STEND_CLUBB']
upwp_C = Dic_C['UPWP_CLUBB']
vpwp_C = Dic_C['VPWP_CLUBB']
thlp2_C = Dic_C['THLP2_CLUBB']
wp2_C = Dic_C['WP2_CLUBB']

wsp_C = np.sqrt( uu_C**2 + vv_C**2 )


In [ ]:
momflx_C = np.sqrt( upwp_C**2 + vpwp_C**2 )

In [ ]:

nt,nz,ny,nx = np.shape( uu_C )
days=np.arange( nt )/4.
zlev=-7.0*np.log( Dic_X['lev']/1_000. )
lat=Dic_X['lat']
lon=Dic_X['lon']


In [ ]:

fig = plt.figure(figsize=(20, 7))
Axes = Pu.axes_def(n=1,nxplo=2,nyplo=1 ) 
ax1 = fig.add_axes( Axes )
ax1.contourf( prl_C[:,35,:] , levels=21)
Axes = Pu.axes_def(n=2,nxplo=2,nyplo=1 ) 
ax1 = fig.add_axes( Axes )
ax1.contourf( utn_C[:,10,35,:] , levels=21)


In [ ]:
print(zlev[10])
print(lat[35])

In [ ]:
k,j=8,35
k,j=8,40
plt.scatter( wsp_C[:,85,j,50:150]-wsp_C[:,92,j,50:150] , utn_C[:,k,j,50:150] ,c=thlp2_C[:,85,j,50:150], s=5 )

In [ ]:
#plt.plot( Dic_C['STEND_CLUBB'][:,90,35,75] ) #, utn_C[:,8,35,50:150] )
plt.plot( np.average( stend_C[:,90,35,:] , axis=1 ) )

In [ ]:
plt.plot( np.average( utn_C[:,8,35,:] , axis=1 ) )

In [ ]:

x=np.average( thlp2_C[:,80,35,:] , axis=1 )
x=np.average( prl_C[:,35,50:100] , axis=1 )
x=np.average( np.average( momflx_C[:,88:,35,50:100] ,axis=1), axis=1 )


y=np.average( utn_C[:,8,35,50:100] , axis=1 )
plt.scatter( x , y )
coef= np.corrcoef( x,y)
print(coef)


In [ ]:
plt.contourf( lat,zlev,np.average( np.average( uu_X, axis=3) , axis=0 ) )

In [ ]:


scale=1
uu_X_x = np.average( uu_X,axis=3)
utgwt_C_x = scale*np.average( utgwt_C ,axis=3)
utgwt_X_x= scale*np.average( utgwt_X ,axis=3)
utgwmm_X_x= scale*np.average( utgwmm_X ,axis=3)
utn_C_x= scale*np.average( utn_C ,axis=3)
utn_X_x= scale*np.average( utn_X ,axis=3)

uu_X_tx = np.average( uu_X_x ,axis=0)
utgwt_C_tx = scale*np.average(utgwt_C_x,axis=0)
utgwt_X_tx= scale*np.average(utgwt_X_x ,axis=0)
utgwmm_X_tx= scale*np.average(utgwmm_X_x ,axis=0)
utn_C_tx= scale*np.average( utn_C_x ,axis=0)
utn_X_tx= scale*np.average( utn_X_x ,axis=0)


In [ ]:

Zl1,Zl0=50.,60.
Zl1,Zl0=30.,50.
lat0,lat1=-70.,-60.
k0=np.argmin( np.abs(  zlev - Zl0 ) )
k1=np.argmin( np.abs(  zlev - Zl1 ) )
ilat0 = np.argmin( np.abs(  Dx2.lat.values - lat0 ) )
ilat1 = np.argmin( np.abs(  Dx2.lat.values - lat1 ) )

print( ilat0 )
print( k0, zlev[k0] )

ndglev=0.1*np.linspace(-100,100,num=21)
fig = plt.figure(figsize=(20, 7))
Axes = Pu.axes_def(n=1,nxplo=3,nyplo=1 ) 
ax1 = fig.add_axes( Axes )
ax1.contourf( days, lat, utn_C_x[:,k0,:].T  , levels=ndglev )
Axes = Pu.axes_def(n=2,nxplo=3,nyplo=1 ) 
ax1 = fig.add_axes( Axes )
ax1.contourf( days, lat, (utgwmm_X_x)[:,k0,:].T   , levels=ndglev/100 )


In [ ]:
plt.plot( utn_C_x[:,k0,20] )
plt.plot( utgwmm_X_x[:,k0,20] )


In [ ]:

fld='Nudge_U'
season ='Sep'
years_c=[2001,2001]
years_x=[2001,2001]
fig = plt.figure(figsize=(20, 7))

#AAyz_x = 86400.*np.average(Dx.Nudge_U[mon,:,:,:],axis=2)
#AAyz_c = 86400.*np.average(D_C.Nudge_U[mon,:,:,:],axis=2)
ndglev=5*np.linspace( -2,2,num=21) 

ndglev = np.array([-100,-50,-20,-10,-5,-4,-3,-2,-1,0,1,2,3,4,5,10,20,50,100]) /5.
ulev=np.linspace(-60,120,num=19)

# Define a custom colormap with linear color transitions
colors = plt.get_cmap('bwr')(np.linspace(0, 1, len(ndglev) - 1))
cmap = Listed_Colormap(colors)

# Create a BoundaryNorm to map values to colors
norm = BoundaryNorm(ndglev, len(ndglev) - 1)

Axes = Pu.axes_def(n=1,nxplo=3,nyplo=1 ) 
ax1 = fig.add_axes( Axes )

co2=ax1.contourf(Dx2.lat, zlev, utn_C_tx ,levels=ndglev,norm=norm,cmap=cmap)
co3=ax1.contour(Dx2.lat, zlev, uu_X_tx ,levels=ulev,colors='black')
ax1.set_ylim( (0,65))
ax1.set_title( f"{season.upper()}  {years_c[0]}-{years_c[-1]} {fld}  {exp_C}" , fontsize=12)

cbar = plt.colorbar(co2, shrink=.6)

Axes = Pu.axes_def(n=2,nxplo=3,nyplo=1 ) 
ax1 = fig.add_axes( Axes )

co2=ax1.contourf(Dx2.lat, zlev, utn_X_tx ,levels=ndglev,norm=norm,cmap=cmap)
co3=ax1.contour(Dx2.lat, zlev, uu_X_tx ,levels=ulev,colors='black')
ax1.set_ylim( (0,65))
ax1.set_title( f"{season.upper()}  {years_c[0]}-{years_c[-1]} {fld} {exp_X}" , fontsize=12)

cbar = plt.colorbar(co2, shrink=.6)

Axes = Pu.axes_def(n=3,nxplo=3,nyplo=1 ) 
ax1 = fig.add_axes( Axes )

co2=ax1.contourf(Dx2.lat, zlev, utn_X_tx-utn_C_tx   ,levels=ndglev,norm=norm,cmap=cmap)
co3=ax1.contour(Dx2.lat, zlev, uu_X_tx   ,levels=ulev,colors='black')
ax1.set_ylim( (0,65))
ax1.set_title(f' {exp_X} - {exp_C}')

cbar = plt.colorbar(co2, shrink=.6)

#fig.suptitle( f'{exp_X}')



In [ ]:
D_C.Nudge_U


In [ ]:
from scipy.ndimage import gaussian_filter

nday=31
ilat=35
klev=8
print(f"Z={zlev[klev]}")
print(Dx2.lat[ilat].values)


Zl1,Zl0=50.,60.
lat0,lat1=-70.,-50.
k0=np.argmin( np.abs(  zlev - Zl0 ) )
k1=np.argmin( np.abs(  zlev - Zl1 ) )
ilat0 = np.argmin( np.abs(  Dx2.lat.values - lat0 ) )
ilat1 = np.argmin( np.abs(  Dx2.lat.values - lat1 ) )

sigma_x , sigma_y =2 , 4

UtN_C = gaussian_filter( 86400.*np.average( np.average(D_C2.Nudge_U[:,k0:k1,ilat0:ilat1,:],axis=1),axis=1) ,sigma=[sigma_x, sigma_y] )
UtN_X = gaussian_filter( 86400.*np.average( np.average(Dx2.Nudge_U[:,k0:k1,ilat0:ilat1,:],axis=1),axis=1) ,sigma=[sigma_x, sigma_y] )
UtGW_X = gaussian_filter( 86400.*np.average( np.average(Dx2.UTEND_GWDTOT[:,k0:k1,ilat0:ilat1,:],axis=1),axis=1) ,sigma=[sigma_x, sigma_y] )
UtGW_mm = gaussian_filter( 86400.*np.average( np.average(Dx2.UTGW_MOVMTN[:,k0:k1,ilat0:ilat1,:],axis=1),axis=1) ,sigma=[sigma_x, sigma_y] )
UtGW_C = gaussian_filter( 86400.*np.average( np.average(D_C2.UTEND_GWDTOT[:,k0:k1,ilat0:ilat1,:],axis=1),axis=1) , sigma=[sigma_x, sigma_y] )
UtPh_C = gaussian_filter( 86400.*np.average( np.average(D_C2.UTEND_PHYSTOT[:,k0:k1,ilat0:ilat1,:],axis=1),axis=1) , sigma=[sigma_x, sigma_y] )
UtPh_X = gaussian_filter( 86400.*np.average( np.average(Dx2.UTEND_PHYSTOT[:,k0:k1,ilat0:ilat1,:],axis=1),axis=1) , sigma=[sigma_x, sigma_y] )

print(f"Z levs ={zlev[k1]} {zlev[k0]}")

fig = plt.figure(figsize=(20, 7))

Axes = Pu.axes_def(n=1,nxplo=3,nyplo=1 ) 
ax1 = fig.add_axes( Axes )
co1=ax1.contourf( D_C2.lon, np.arange(nday*4), UtN_C-0*UtN_X ,levels=np.linspace(-40,40,num=21), cmap='bwr' )
plt.colorbar(co1,ax=ax1)

Axes = Pu.axes_def(n=2,nxplo=3,nyplo=1 ) 
ax1 = fig.add_axes( Axes )
co1=ax1.contourf( D_C2.lon, np.arange(nday*4),  UtGW_mm ,levels=np.linspace(-20,20,num=21), cmap='bwr', extend='both' )
plt.colorbar(co1,ax=ax1)

Axes = Pu.axes_def(n=3,nxplo=3,nyplo=1 ) 
ax1 = fig.add_axes( Axes )
co1=ax1.contourf( D_C2.lon, np.arange(nday*4), UtPh_X - UtPh_C ,levels=np.linspace(-20,20,num=21), cmap='bwr', extend='both' )
plt.colorbar(co1,ax=ax1)



In [ ]:
season='SON'
fld='Nudge_U'
#fld='UTEND_GWDTOT'
scale=86400.

AAs_C,years_c,months_c = Av.Seasonal( ds=D_C, season=season , fld=fld ,return_time=True)
AAs_X,years_x,months_x = Av.Seasonal( ds=Dx, season=season , fld=fld ,return_time=True)
UUs_X,years_x,months_x = Av.Seasonal( ds=Dx, season=season , fld='U' ,return_time=True)

AAs_C = scale*AAs_C
AAs_X = scale*AAs_X

AAsx_C=np.average( AAs_C , axis=2 )
AAsx_X=np.average( AAs_X , axis=2 )
UUsx_X=np.average( UUs_X , axis=2 )


#kl0,kl1=9,10 #11,12
kl0,kl1=11,12
kl0,kl1=7,9
#kl0,kl1=52,60


In [ ]:


fig = plt.figure(figsize=(20, 7))

#AAyz_x = 86400.*np.average(Dx.Nudge_U[mon,:,:,:],axis=2)
#AAyz_c = 86400.*np.average(D_C.Nudge_U[mon,:,:,:],axis=2)
ndglev=5*np.linspace( -2,2,num=21) 

ndglev = np.array([-100,-50,-20,-10,-5,-4,-3,-2,-1,0,1,2,3,4,5,10,20,50,100]) /5.
ulev=np.linspace(-60,120,num=19)

# Define a custom colormap with linear color transitions
colors = plt.get_cmap('bwr')(np.linspace(0, 1, len(ndglev) - 1))
cmap = ListeD_Colormap(colors)

# Create a BoundaryNorm to map values to colors
norm = BoundaryNorm(ndglev, len(ndglev) - 1)

Axes = Pu.axes_def(n=1,nxplo=3,nyplo=1 ) 
ax1 = fig.add_axes( Axes )

co2=ax1.contourf(Dx.lat, zlev, AAsx_C ,levels=ndglev,norm=norm,cmap=cmap)
co3=ax1.contour(Dx.lat, zlev, UUsx_X ,levels=ulev,colors='black')
ax1.set_ylim( (0,65))
ax1.set_title( f"{season.upper()}  {years_c[0]}-{years_c[-1]} {fld}  {exp_C}" , fontsize=12)

cbar = plt.colorbar(co2, shrink=.6)

Axes = Pu.axes_def(n=2,nxplo=3,nyplo=1 ) 
ax1 = fig.add_axes( Axes )

co2=ax1.contourf(Dx.lat, zlev, AAsx_X ,levels=ndglev,norm=norm,cmap=cmap)
co3=ax1.contour(Dx.lat, zlev, UUsx_X ,levels=ulev,colors='black')
ax1.set_ylim( (0,65))
ax1.set_title( f"{season.upper()}  {years_c[0]}-{years_c[-1]} {fld} {exp_X}" , fontsize=12)

cbar = plt.colorbar(co2, shrink=.6)

Axes = Pu.axes_def(n=3,nxplo=3,nyplo=1 ) 
ax1 = fig.add_axes( Axes )

co2=ax1.contourf(Dx.lat, zlev, AAsx_X-AAsx_C ,levels=ndglev,norm=norm,cmap=cmap)
co3=ax1.contour(Dx.lat, zlev, UUsx_X ,levels=ulev,colors='black')
ax1.set_ylim( (0,65))
ax1.set_title(f' {exp_X} - {exp_C}')

cbar = plt.colorbar(co2, shrink=.6)

#fig.suptitle( f'{exp_X}')



In [ ]:


Zl1,Zl0=49.,55.

kl0=np.argmin( np.abs(  zlev - Zl0 ) )
kl1=np.argmin( np.abs(  zlev - Zl1 ) )

AAsz_C = np.average( AAs_C[kl0:kl1,:,:],axis=0)
AAsz_X = np.average( AAs_X[kl0:kl1,:,:],axis=0)




######################
# Pretty latlon plot
######################

#MapProj = ccrs.Robinson(central_longitude=180.)
MapProj = ccrs.PlateCarree(central_longitude=180.)
DataProj = ccrs.PlateCarree()

fig = plt.figure(figsize=(20, 12))
   
ndglev=10*np.linspace( -2,2,num=21) /1.
npo=0

zA = f" ( {Dx.lev[kl0]:.0f}-{Dx.lev[kl1]:.0f} hPa, {zlev[kl1]:.0f}-{zlev[kl0]:.0f} km)"
CoastColors=['black','black','black','black','black','black']

npo=npo+1
Axes = Pu.axes_def(n=npo,nxplo=2,nyplo=2 ) 
ax1 = fig.add_axes( Axes , projection=MapProj)
ax1.set_global()
ax1.coastlines(resolution='110m',color=CoastColors[npo-1])
co2=ax1.contourf(Dx.lon,Dx.lat, AAsz_C ,transform=DataProj,levels=ndglev,cmap='bwr', extend='both')
cbar = plt.colorbar(co2, shrink=.6)
ax1.set_title( f"{season.upper()}  {years_c[0]}-{years_c[-1]} {fld} {zA}  {exp_C}" , fontsize=16)

npo=npo+1
Axes = Pu.axes_def(n=npo,nxplo=2,nyplo=2 ) 
ax1 = fig.add_axes( Axes , projection=MapProj)
ax1.set_global()
ax1.coastlines(resolution='110m',color=CoastColors[npo-1])
co2=ax1.contourf(Dx.lon,Dx.lat, AAsz_X ,transform=DataProj,levels=ndglev,cmap='bwr', extend='both')
cbar = plt.colorbar(co2, shrink=.6)
ax1.set_title( f"{season.upper()}  {years_x[0]}-{years_x[-1]} {fld} {zA}  {exp_X}" , fontsize=16)

npo=npo+1

if (fld == 'Nudge_U'):
    Axes = Pu.axes_def(n=npo,nxplo=2,nyplo=2 ) 
    print(f" {npo} {Axes} ")
    ax1 = fig.add_axes( Axes , projection=MapProj)
    ax1.set_global()
    ax1.coastlines(resolution='110m',color=CoastColors[npo-1])
    co2=ax1.contourf(Dx.lon,Dx.lat, np.abs(AAsz_C) - np.abs(AAsz_X) ,transform=DataProj,levels=ndglev ,cmap='bwr_r', extend='both')
    cbar = plt.colorbar(co2, ax=ax1 , shrink=0.6 )# , extend='both' ,extendfrac=0.1)
    #ax1.set_title( f"{season.upper()}  {years_c[0]}-{years_c[-1]} Nudge_U {zA} Diff " , fontsize=16)
    ax1.set_title(f' ||Test|| - ||Control||')
    
    ax1.text(1.15, 0.1, f"{exp_C} was better", fontsize=12, ha='left', va='center', 
                bbox=dict(facecolor='white', edgecolor='black') , 
                transform=ax1.transAxes, rotation=0.)  #xycoords='axes fraction')
    ax1.text(1.15, 0.9, f"{exp_X} is better", fontsize=12, ha='left', va='center', 
                bbox=dict(facecolor='white', edgecolor='black') , 
                transform=ax1.transAxes, rotation=0.)  #xycoords='axes fraction')
    
    ax1.annotate('', xy=(1.2, 0.8), xytext=(1.2, 0.6), 
                 arrowprops=dict(arrowstyle='->', lw=2),
                 xycoords='axes fraction', textcoords='axes fraction',fontsize=12)
    
    ax1.annotate('', xy=(1.2, 0.2), xytext=(1.2, 0.4), 
                 arrowprops=dict(arrowstyle='->', lw=2),
                 xycoords='axes fraction', textcoords='axes fraction',fontsize=12)
    
else:
    Axes = Pu.axes_def(n=npo,nxplo=2,nyplo=2 ) 
    print(f" {npo} {Axes} ")
    ax1 = fig.add_axes( Axes , projection=MapProj)
    ax1.set_global()
    ax1.coastlines(resolution='110m',color=CoastColors[npo-1])
    co2=ax1.contourf(Dx.lon,Dx.lat, AAsz_X - AAsz_C ,transform=DataProj,levels=ndglev/2,cmap='bwr', extend='both')
    cbar = plt.colorbar(co2, ax=ax1 , shrink=0.6 )# , extend='both' ,extendfrac=0.1)
    #ax1.set_title( f"{season.upper()}  {years_c[0]}-{years_c[-1]} Nudge_U {zA} Diff " , fontsize=16)
    ax1.set_title(f' Test - Control')


fig.suptitle(  f"Control = {exp_C}, Test = {exp_X} ", ha="left", fontsize=16, x=0.0 )


In [ ]:
#################











##############